## building an agent that interacts with a search engine 
- ask agent questions 
- watch agent call the search tool 
- have conversations with the agent

[python.langchain tutorial](https://python.langchain.com/docs/tutorials/agents/)

In [13]:
import getpass
import os
from dotenv import load_dotenv

# os.environ["LANGMSITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

# loads variables from .env
load_dotenv()

tracing = os.getenv("LANGSMITH_TRACING", "false")
api_key = os.getenv("LANGSMITH_API_KEY")

print("Tracing:", tracing)
print("API Key is set:", api_key is not None)

tav_key = os.getenv("TAVILY_API_KEY")
print("Tavily API Key is set:", tav_key is not None)
# os.environ["TAVLIY_API_KEY"] = getpass.getpass()

gemini_api = os.getenv("GOOGLE_API_KEY")
print("Google Gemini API Key is set")

Tracing: true
API Key is set: True
Tavily API Key is set: True
Google Gemini API Key is set


**note: Tavily researcher plan gives 1000 API credits (monthly) for free, no credit card required**

In [12]:
# we need to create the tools we want to use
# main tool of choice will be Tavily (a search engine)
# we can use the dedicated langchain-tavily integration
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF?")
print(search_results)
# if we want, we can create other tools 
# once we have all the tools we want, we can put them in a list that we will reference later
tools = [search]

{'query': 'What is the weather in SF?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1751311975, 'localtime': '2025-06-30 12:32'}, 'current': {'last_updated_epoch': 1751311800, 'last_updated': '2025-06-30 12:30', 'temp_c': 14.3, 'temp_f': 57.7, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 10.5, 'wind_kph': 16.9, 'wind_degree': 251, 'wind_dir': 'WSW', 'pressure_mb': 1014.0, 'pressure_in': 29.93, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 94, 'cloud': 50, 'feelslike_c': 13.1, 'feelslike_f': 55.5, 'windchill_c': 12.5, 'windchill_f': 54.5, 'heatindex_c': 13.3, 'heatindex_f': 56.0, 'dewpoint_c': 11.8,

In [16]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [1]:
# import relevant functionality 
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

packages i need to install in virtual environment
```
pip install langchain
pip insall langgraph
pip install langchain-community tavily-python
pip install -U langchain-anthropic
pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite
pip install -U langchain-google-genai
```

confirmed i have the correct packages:
```
pip show langgraph
pip show langchain-community
pip show tavily-python
```

<span style="color: yellow;">ensure you have a Tavily API key set as TAVILY_API_KEY in the environment</span>

In [15]:
# create the agent 
memory = MemorySaver()
# i think this is free ? 
model = init_chat_model("anthropic:claude-3-5-sonnet-latest")
search = TavilySearch(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [ ]:
# use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user", 
    "content": "Hi, I'm Bob and I live in SF.",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

SyntaxError: expression expected after dictionary key and ':' (3376947980.py, line 6)